# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
# pull the weather csv
# index adjusted with code from https://stackoverflow.com/questions/36519086/how-to-get-rid-of-unnamed-0-column-in-a-pandas-dataframe
weather_df = pd.read_csv("../output_data/cities.csv", index_col=[0])
weather_df.head()

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,bredasdorp,ZA,-34.5322,20.0403,55.40,55.40,82,2,4.61
1,port lincoln,CD,-7.0349,29.7638,72.30,72.30,71,65,8.08
2,mataura,NZ,-46.1927,168.8643,49.30,49.30,78,97,6.71
3,akcakoca,TR,41.0866,31.1162,34.75,34.75,86,53,5.93
4,ponta do sol,PT,38.7333,-28.2667,58.98,58.98,98,100,28.74


In [19]:
weather_df["Max Temp (°F)"].max()

89.01

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
gmaps.configure(api_key=g_key)

In [21]:
# store lat lng into Locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# create and use humidity as weight
humidity = weather_df["Humidity (%)"].astype(float)

In [22]:
# create humidity heat map
humidity_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                dissipating = False, max_intensity = 100,
                                point_radius = 1)

humidity_fig.add_layer(heat_layer)

humidity_fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
# creating df for ideal weather conditions
weather_ideal_df = weather_df.loc[(weather_df['Max Temp (°F)'] >= 75) & 
                                  (weather_df['Wind Speed (mph)'] <= 15) &
                                  (weather_df['Cloudiness (%)'] <= 33) & 
                                  (weather_df['Humidity (%)'] >= 66) & 
                                  (weather_df['Humidity (%)'] <= 90)].dropna()
#index successfully reset with code from https://stackoverflow.com/questions/27736267/pandas-reset-index-is-not-taking-effect
weather_ideal_df.reset_index(drop=True, inplace=True)
weather_ideal_df

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,saint-pierre,RE,-21.3393,55.4781,74.46,77.00,69,0,8.05
1,arraial do cabo,BR,-22.8362,-46.6460,75.76,78.01,78,0,6.91
2,anloga,GH,5.7947,0.8973,82.40,82.40,83,20,9.22
3,caravelas,BR,-17.7125,-39.2481,76.46,76.46,83,0,6.24
4,carnarvon,AU,-24.8667,113.6333,75.20,75.20,73,0,12.66
5,georgetown,MY,5.4112,100.3354,74.93,77.00,88,20,4.61
6,badou,TG,7.5833,0.6000,75.20,75.20,71,9,2.04
7,atakpame,TG,7.5333,1.1333,77.50,77.50,71,29,5.32
8,winneba,GH,5.3511,-0.6231,82.40,82.40,83,0,9.22
9,lazaro cardenas,DO,19.4558,-71.3261,75.00,75.00,79,3,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = weather_ideal_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,74.46,77.00,69,0,8.05,
1,arraial do cabo,BR,-22.8362,-46.6460,75.76,78.01,78,0,6.91,
2,anloga,GH,5.7947,0.8973,82.40,82.40,83,20,9.22,
3,caravelas,BR,-17.7125,-39.2481,76.46,76.46,83,0,6.24,
4,carnarvon,AU,-24.8667,113.6333,75.20,75.20,73,0,12.66,


In [ ]:
city_coordinates = f"Latitude": (hotel_df['Latitude']), 
                      "Longitude": (hotel_df['Longitude'])}
target_coordinates

In [ ]:
for city in b:
    params = {
        "location": ,
        "keyword": ,
        "radius": ,
        "type": ,
        "key": }

target_type = "lodging",
target_radius = 500, 
target_type

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
